# Assignment 3

Import libraries and define common helper functions

In [1]:
pip install fastavro==1.5.1

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install iteration_utilities

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install avro

Note: you may need to restart the kernel to use updated packages.


In [4]:
from iteration_utilities import unique_everseen
import os
import sys
import gzip
import json
from pathlib import Path
import csv
import avro.schema
import fastavro
import pandas as pd
import s3fs
# from fs_s3fs import S3FS
import pyarrow as pa
from pyarrow.json import read_json
import pyarrow.parquet as pq
#import fastavro
from fastavro.schema import load_schema
from fastavro import writer, reader, parse_schema
#import pygeohashpisr d
import pygeohash
import snappy
import jsonschema
from jsonschema import validate

from jsonschema.exceptions import ValidationError


endpoint_url='/home/jovyan/'

current_dir = Path(os.getcwd()).absolute()
schema_dir = current_dir.joinpath('schemas')
results_dir = current_dir.joinpath('results')
results_dir.mkdir(parents=True, exist_ok=True)

print(current_dir)
print(schema_dir)
print(results_dir)

def read_jsonl_data():
    f_gz = '/home/jovyan/assignment03_Vayuvegula_SomaShekar/data/processed/openflights/routes.jsonl.gz'
    with gzip.open(f_gz, 'rb') as f:
        records = [json.loads(line) for line in f.readlines()]

    return records

/home/jovyan/assignment03_Vayuvegula_SomaShekar
/home/jovyan/assignment03_Vayuvegula_SomaShekar/schemas
/home/jovyan/assignment03_Vayuvegula_SomaShekar/results


Load the records from https://storage.budsc.midwest-datascience.com/data/processed/openflights/routes.jsonl.gz 

In [5]:
records = read_jsonl_data()

## 3.1

### 3.1.a JSON Schema

In [6]:
def validate_jsonl_data(records):
    schema_path = schema_dir.joinpath('routes-schema.json')
    with open(schema_path) as f:
        schema = json.load(f)
    validation_csv_path = results_dir.joinpath('validation-results.csv')
    # Open the validation CSV
    with open(validation_csv_path, 'w', encoding="utf-8") as f:
        # Create column names
        fieldnames = ['row', 'valid', 'msg']
        # Assign CSV writer object
        csv_writer = csv.DictWriter(f, fieldnames=fieldnames, lineterminator = '\n')
        csv_writer.writeheader()   
        for i, record in enumerate(records):
            try:
                ## TODO: Validate record 
                result = dict(
                    row = i,
                    valid = True,
                    msg = record
                )
                pass
            except ValidationError as e:
                ## Print message if invalid record
                result = dict(
                    row = i,
                    valid = False,
                    msg = record
                )
                pass
            finally:
                # Write the line to the CSV.
                csv_writer.writerow(result)

validate_jsonl_data(records)

### 3.1.b Avro

In [7]:
def create_avro_dataset(records):
    schema_path = schema_dir.joinpath('routes.avsc')
    data_path = results_dir.joinpath('routes.avro')
    ## TODO: Use fastavro to create Avro dataset
    with open(schema_path) as f:
        schema = json.load(f)  
    with open(data_path, 'wb') as out:
        fastavro.writer(out, schema, records)
        
create_avro_dataset(records)

### 3.1.c Parquet

In [8]:
def create_parquet_dataset():
    src_data_path = '/home/jovyan/assignment03_Vayuvegula_SomaShekar/data/processed/openflights/routes.jsonl.gz'
    parquet_output_path = results_dir.joinpath('routes.parquet')
    """ 
    s3 = s3fs.S3FileSystem(
        anon=True,
        client_kwargs={
            'endpoint_url': endpoint_url
        }
    )
    """

    with gzip.open(src_data_path, 'rb') as f:
        records = [json.loads(line) for line in f.readlines()]
    print(records)
    df = pd.DataFrame(records)
    ## TODO: Use Apache Arrow to create Parquet table and save the dataset
    table = pa.Table.from_pandas(df)
    print(table)
    pq.write_table(table, parquet_output_path, compression='none')

create_parquet_dataset()

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



pyarrow.Table
airline: struct<active: bool, airline_id: int64, alias: string, callsign: string, country: string, iata: string, icao: string, name: string>
  child 0, active: bool
  child 1, airline_id: int64
  child 2, alias: string
  child 3, callsign: string
  child 4, country: string
  child 5, iata: string
  child 6, icao: string
  child 7, name: string
src_airport: struct<airport_id: int64, altitude: int64, city: string, country: string, dst: string, iata: string, icao: string, latitude: double, longitude: double, name: string, source: string, timezone: double, type: string, tz_id: string>
  child 0, airport_id: int64
  child 1, altitude: int64
  child 2, city: string
  child 3, country: string
  child 4, dst: string
  child 5, iata: string
  child 6, icao: string
  child 7, latitude: double
  child 8, longitude: double
  child 9, name: string
  child 10, source: string
  child 11, timezone: double
  child 12, type: string
  child 13, tz_id: string
dst_airport: struct<airport_id: 

### 3.1.d Protocol Buffers

In [9]:
sys.path.insert(0, os.path.abspath('routes_pb2'))

import routes_pb2

def _airport_to_proto_obj(airport):
    obj = routes_pb2.Airport()
    if airport is None:
        return None
    if airport.get('airport_id') is None:
        return None

    obj.airport_id = airport.get('airport_id')
    if airport.get('name'):
        obj.name = airport.get('name')
    if airport.get('city'):
        obj.city = airport.get('city')
    if airport.get('iata'):
        obj.iata = airport.get('iata')
    if airport.get('icao'):
        obj.icao = airport.get('icao')
    if airport.get('altitude'):
        obj.altitude = airport.get('altitude')
    if airport.get('timezone'):
        obj.timezone = airport.get('timezone')
    if airport.get('dst'):
        obj.dst = airport.get('dst')
    if airport.get('tz_id'):
        obj.tz_id = airport.get('tz_id')
    if airport.get('type'):
        obj.type = airport.get('type')
    if airport.get('source'):
        obj.source = airport.get('source')

    obj.latitude = airport.get('latitude')
    obj.longitude = airport.get('longitude')

    return obj


def _airline_to_proto_obj(airline):
    obj = routes_pb2.Airline()
    ## TODO: Create an Airline obj using Protocol Buffers API
    if airline is None:
        return None
    if airline.get('airline_id') is None:
        return None

    obj.airline_id = airline.get('airline_id')
    if airline.get('name'):
        obj.name = airline.get('name')
    if airline.get('alias'):
        obj.alias = airline.get('alias')
    if airline.get('iata'):
        obj.iata = airline.get('iata')
    if airline.get('icao'):
        obj.icao = airline.get('icao')
    if airline.get('callsign'):
        obj.callsign = airline.get('callsign')
    if airline.get('country'):
        obj.country = airline.get('country')
    if airline.get('active'):
        obj.active = airline.get('active')
        
    return obj


def create_protobuf_dataset(records):
    routes = routes_pb2.Routes()
    for record in records:
        route = routes_pb2.Route()
        ## TODO: Implement the code to create the Protocol Buffers Dataset
        for key, value in record.items():
            if key=='airline':
                airline = _airline_to_proto_obj(value)
                airin = route.airline
                airin.name = airline.name
                airin.airline_id = airline.airline_id
                airin.active = airline.active
            if key=='src_airport' and value is not None:
                src_airport = _airport_to_proto_obj(value)
                srcairin = route.src_airport
                srcairin.name = src_airport.name
                srcairin.airport_id = src_airport.airport_id
                srcairin.latitude = src_airport.latitude
                srcairin.longitude = src_airport.longitude

            if key=='dst_airport' and value is not None:
                dst_airport = _airport_to_proto_obj(value)
                dstairin = route.dst_airport
                dstairin.name = dst_airport.name
                dstairin.airport_id = dst_airport.airport_id
                dstairin.latitude = dst_airport.latitude
                dstairin.longitude = dst_airport.longitude

            if key=='codeshare':
                route.codeshare = value

        routes.route.append(route)

    data_path = results_dir.joinpath('routes.pb')

    with open(data_path, 'wb') as f:
        f.write(routes.SerializeToString())
        
    compressed_path = results_dir.joinpath('routes.pb.snappy')
    
    with open(compressed_path, 'wb') as f:
        f.write(snappy.compress(routes.SerializeToString()))
        
create_protobuf_dataset(records)

## 3.2

### 3.2.a Simple Geohash Index

In [10]:
def create_hash_dirs(records):
    geoindex_dir = results_dir.joinpath('geoindex')
    geoindex_dir.mkdir(exist_ok=True, parents=True)
    hashes = []
    for record in records:
        src_airport = record.get('src_airport', {})
        if src_airport:
            latitude = src_airport.get('latitude')
            longitude = src_airport.get('longitude')
            if latitude and longitude:
                ## TODO: use pygeohash.encode() to assign geohashes to the records and complete the hashes list
                hashes.append(pygeohash.encode(longitude=longitude,latitude=latitude,precision=3))
                record['geohash'] = pygeohash.encode(longitude=longitude,latitude=latitude,precision=3)
    hashes.sort()
    three_letter = sorted(list(set([entry[:3] for entry in hashes])))
    hash_index = {value: [] for value in three_letter}
    for record in records:
        geohash = record.get('geohash')
        if geohash:
            hash_index[geohash[:3]].append(record)
    for key, values in hash_index.items():
        output_dir = geoindex_dir.joinpath(str(key[:1])).joinpath(str(key[:2]))
        output_dir.mkdir(exist_ok=True, parents=True)
        output_path = output_dir.joinpath('{}.jsonl.gz'.format(key))
        with gzip.open(output_path, 'w') as f:
            json_output = '\n'.join([json.dumps(value) for value in values])
            f.write(json_output.encode('utf-8'))
            
create_hash_dirs(records)

### 3.2.b Simple Search Feature

In [11]:
def airport_search(latitude, longitude):
    src_geohash_code = pygeohash.encode(longitude=longitude,latitude=latitude,precision=15)
    # load the data from that file
    geoindex_dir = results_dir.joinpath('geoindex')
    # building my search area
    my_geo_dir = geoindex_dir.joinpath(str(src_geohash_code[0]))
    my_geo_dir = my_geo_dir.joinpath(str(src_geohash_code[:2]))
    my_geo_dir = my_geo_dir.joinpath(str(src_geohash_code[:3])+".jsonl.gz")

    # print(my_geo_dir)
    with open(my_geo_dir, 'rb') as f_gz:
        with gzip.open(f_gz, 'rb') as f:
            geo_records = [json.loads(line) for line in f.readlines()]
            current_dis = 0
            closest_airport = {}
            for distance in geo_records:
                if distance['src_airport']:
                    temp_lat = distance["src_airport"]["latitude"]
                    temp_lon = distance["src_airport"]["longitude"]
                    dst_geohash_code = pygeohash.encode(longitude=temp_lon,latitude=temp_lat,precision=15)
                    dis = pygeohash.geohash_approximate_distance(str(src_geohash_code), str(dst_geohash_code))
                    if current_dis == 0 or dis <= current_dis:
                        current_dis = dis
                        closest_airport = distance["airline"]["name"]

            print("The closest airport is:" + closest_airport + ", which is: "+ str(current_dis) + " km away")

airport_search(41.1499988, -95.91779)

The closest airport is:Southwest Airlines, which is: 19545 km away
